In [ ]:
#import library

import time
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

import mnist_reader
import CNNModel

In [ ]:
# coding:utf-8

import time
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

import mnist_reader
import CNNModel

def main():
    # load the data
    X_train, y_train = mnist_reader.load_mnist('data', kind='train')
    X_test, y_test = mnist_reader.load_mnist('data', kind='t10k')    
    print("Data load finished")
    
    # data pre-processing
    # rescale the data to 0-1
    X_train = X_train / 255
    X_test = X_test / 255

    # KNN
    print("="*80)
    print("KNN model")
    
    # initilize models
    knn = KNeighborsClassifier(weights='distance', p=1, algorithm='brute')
    
    # model train
    time_start = time.time()
    knn.fit(X_train, y_train)
    time_end = time.time()
    print("KNN's training takes {:.3} seconds".format(time_end - time_start))

    # model save
    joblib.dump(knn, 'knn.model')
    print("KNN saved")
    

    # validate
    time_start = time.time()
    y_pred = knn.predict(X_test)
    time_end = time.time()
    print("KNN's prediction takes {:.5} seconds".format(time_end - time_start))
    
    # metrics
    print("Train acc = ", knn.score(X_train, y_train))
    print("Acc = ", metrics.accuracy_score(y_test, y_pred))
    print("Pre = ", metrics.precision_score(y_test, y_pred, average='macro'))
    print("Recall = ", metrics.recall_score(y_test, y_pred, average='macro'))
    
    # confusion matrix
    print("Confusion matrix")
    print(metrics.confusion_matrix(y_test, y_pred))
    print()
    
    # corss validation
    kf = KFold(n_splits=10, shuffle=True)
    acc_list = []
    pre_list = []
    recall_list = []
    for train_ind, test_ind in kf.split(X_train):
        knn = KNeighborsClassifier(weights='distance', p=1, algorithm='brute')
        knn.fit(X_train[train_ind], y_train[train_ind])
        y_pred = knn.predict(X_train[test_ind])
        acc_list.append(metrics.accuracy_score(y_train[test_ind], y_pred))
        pre_list.append(metrics.precision_score(y_train[test_ind], y_pred, average='macro'))
        recall_list.append(metrics.recall_score(y_train[test_ind], y_pred, average='macro'))
    print("10 Fold corss validation")
    print("Mean accuracy = ", np.array(acc_list).mean())
    print("Mean precision = ", np.array(pre_list).mean())
    print("Mean recall = ", np.array(recall_list).mean())
    
    
    # CNN
    print("="*80)
    print("CNN model")
    # initilize
    cnn = CNNModel.CNNModel()
    cnn.initilize()
    
    # train
    time_start = time.time()
    cnn.fit(X_train, y_train, X_test, y_test)
    time_end = time.time()
    print("CNN's training takes {:.3} seconds".format(time_end - time_start))    
    
    # model save
    cnn.save()
    print("CNN saved")


    # validate
    time_start = time.time()
    y_pred = cnn.predict(X_test)
    time_end = time.time()
    print("CNN's prediction takes {:.5} seconds".format(time_end - time_start))    
    
    # metrics
    y_train_pred = cnn.predict(X_train)
    print("Train acc = ", metrics.accuracy_score(y_train, y_train_pred))
    print("Acc = ", metrics.accuracy_score(y_test, y_pred))
    print("Pre = ", metrics.precision_score(y_test, y_pred, average='macro'))
    print("Recall = ", metrics.recall_score(y_test, y_pred, average='macro'))    
    
    # confusion matrix
    print("Confusion matrix")
    print(metrics.confusion_matrix(y_test, y_pred))
    print()
    
    # corss validation
    kf = KFold(n_splits=10, shuffle=True)
    acc_list = []
    pre_list = []
    recall_list = []
    for train_ind, test_ind in kf.split(X_train):
        cnn = CNNModel.CNNModel()
        cnn.initilize()        
        cnn.fit(X_train[train_ind], y_train[train_ind], X_train[test_ind], y_train[test_ind])
        y_pred = cnn.predict(X_train[test_ind])
        acc_list.append(metrics.accuracy_score(y_train[test_ind], y_pred))
        pre_list.append(metrics.precision_score(y_train[test_ind], y_pred, average='macro'))
        recall_list.append(metrics.recall_score(y_train[test_ind], y_pred, average='macro'))
    print("10 Fold corss validation")
    print("Mean accuracy = ", np.array(acc_list).mean())
    print("Mean precision = ", np.array(pre_list).mean())
    print("Mean recall = ", np.array(recall_list).mean())    


    # Random Forest
    print("="*80)
    print("Random Forest model")    
    
    # initilize
    rf = RandomForestClassifier(n_estimators=55, max_depth=20)
    
    # train
    time_start = time.time()
    rf.fit(X_train, y_train)
    time_end = time.time()
    print("RF's training takes {:.3} seconds".format(time_end - time_start))        
    
    # model save
    joblib.dump(rf, 'rf.model')
    print("RandomForest saved")

 
    # validate
    time_start = time.time()
    y_pred = rf.predict(X_test)
    time_end = time.time()
    print("RF's prediction takes {:.3} seconds".format(time_end - time_start))          
    
    # metrics
    print("Train acc = ", rf.score(X_train, y_train))
    print("Acc = ", metrics.accuracy_score(y_test, y_pred))
    print("Pre = ", metrics.precision_score(y_test, y_pred, average='macro'))
    print("Recall = ", metrics.recall_score(y_test, y_pred, average='macro'))        
    
    # confusion matrix
    print("Confusion matrix")
    print(metrics.confusion_matrix(y_test, y_pred))
    print()
    
    # corss validation
    kf = KFold(n_splits=10, shuffle=True)
    acc_list = []
    pre_list = []
    recall_list = []
    for train_ind, test_ind in kf.split(X_train):
        rf = RandomForestClassifier(n_estimators=55, max_depth=20)
        rf.fit(X_train[train_ind], y_train[train_ind])
        y_pred = rf.predict(X_train[test_ind])
        acc_list.append(metrics.accuracy_score(y_train[test_ind], y_pred))
        pre_list.append(metrics.precision_score(y_train[test_ind], y_pred, average='macro'))
        recall_list.append(metrics.recall_score(y_train[test_ind], y_pred, average='macro'))
    print("10 Fold corss validation")
    print("Mean accuracy = ", np.array(acc_list).mean())
    print("Mean precision = ", np.array(pre_list).mean())
    print("Mean recall = ", np.array(recall_list).mean())    

    

def test_model_load():
    # load the test data
    X_test, y_test = mnist_reader.load_mnist('data', kind='t10k')    
    print("Data load finished")
    
    # data pre-processing
    # rescale the data to 0-1
    X_test = X_test / 255

    # load model
#    knn = joblib.load('knn.model')
#    rf = joblib.load('rf.model')
#    cnn = CNNModel.CNNModel()
#    cnn.load()
#    print("Model load finished")

    X_test = X_test[:1000]
    y_test = y_test[:1000]

    # predict
    knn_pred = knn.predict(X_test)
    cnn_pred = cnn.predict(X_test)
    rf_pred = rf.predict(X_test)

    # accuracy
    print("KNN Acc = ", metrics.accuracy_score(y_test, knn_pred))
    print("CNN Acc = ", metrics.accuracy_score(y_test, cnn_pred))
    print("RF  Acc = ", metrics.accuracy_score(y_test, rf_pred))


if __name__ == '__main__':
    # train the model
    # main()

    # load and test
    test_model_load()
    